## Wine Quality Classifier (Model Deplyoment and Packaging):
* [Read Model from MLFlow Server](#read-model-mlflow)
* [Model Loaded Properly - Prediction on 6 DataPoints](#loaded-model-performance)
* [Local MLFlow Model Server](#local-model-server)
* [Model Packaging using Docker](#model-packaging)
* [MLFlow Model Server on OpenShift](#remote-model-server)

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import mlflow
from mlflow.tracking import MlflowClient

from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix

import requests

import warnings
warnings.filterwarnings('ignore')

<a id="read-model-mlflow"></a>
# Read Model from MLFlow Server

In [32]:
# Log into MLflow
client = MlflowClient()

# Use the MLFlow instance deployed on the openshift cluster
mlflow.set_tracking_uri(uri="http://mlflow-mlflow.apps.cluster-db46l.dynamic.redhatworkshops.io/")

In [33]:
# Load Model from mlflow in loaded_model
MODEL_NAME = "ElasticnetWineModel"
MODEL_VERSION= 3
MODEL_URI = f"models:/{MODEL_NAME}/{MODEL_VERSION}"
loaded_model = mlflow.pyfunc.load_model(MODEL_URI)

<a id="loaded-model-performance"></a>
# Model Loaded Properly - Prediction on 6 DataPoints

In [34]:
# Load testing data and assign the X_test and y_test
test_df = pd.read_csv('testing_data/test_dataset.csv')

# Test Model Performance on 6 data points from testing_data/test_dataset.csvt
test_df_6 = test_df.head(6)

# Prepare the test input by dropping the 'best quality' columns
test_df_6_input = test_df_6.drop(['best quality'], axis=1)

# Extract the actual Wine Quality for the first 6 examples from test_dataset
actual_class_test = test_df_6['best quality']

# Use the trained model to predict the class for the test input
predicted_class_test = pd.DataFrame(loaded_model.predict(test_df_6_input), columns=['Predicted Wine Quality'])

# Combine predicted and actual classes into a single DataFrame
model_output = pd.concat([predicted_class_test, actual_class_test.reset_index(drop=True)], axis=1)

# Rename columns for clarity
model_output.columns = ['Predicted Wine Quality', 'Actual Wine Quality']

print(model_output)

   Predicted Wine Quality  Actual Wine Quality
0                       0                    1
1                       1                    1
2                       1                    1
3                       1                    1
4                       0                    0
5                       1                    1


<a id="local-model-server"></a>
# Local MLFlow Model Server

### Run this commands in git bash to 
* add the MLFlow instance on the OpenShift as tracking server
* and start a model server locally

```
export MLFLOW_TRACKING_URI=http://mlflow-mlflow.apps.cluster-db46l.dynamic.redhatworkshops.io
```

```
mlflow models serve -m "models:/MODEL_NAME/MODEL_VERSION" --env-manager local --no-conda
i.e mlflow models serve -m "models:/ElasticnetWineModel/1" --env-manager local --no-conda
```

when it is running, we can send prediction requests to the endpoint: *"http://127.0.0.1:5000/invocations"*, as shown below.

In [35]:
# Prepare the inference input 
test_df_6_input.to_dict(orient ='split')

{'index': [0, 1, 2, 3, 4, 5],
 'columns': ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol'],
 'data': [[8.6, 0.23, 0.46, 1.0, 0.054, 9.0, 72.0, 0.9941, 2.95, 0.49, 9.1],
  [6.0, 0.18, 0.27, 1.5, 0.089, 40.0, 143.0, 0.9923, 3.49, 0.62, 10.8],
  [7.2, 0.21, 0.37, 1.6, 0.049, 23.0, 94.0, 0.9924, 3.16, 0.48, 10.9],
  [6.1, 0.255, 0.44, 12.3, 0.045, 53.0, 197.0, 0.9967, 3.24, 0.54, 9.5],
  [6.4, 0.24, 0.25, 20.2, 0.083, 35.0, 157.0, 0.99976, 3.17, 0.5, 9.1],
  [7.3, 0.33, 0.4, 6.85, 0.038, 32.0, 138.0, 0.992, 3.03, 0.3, 11.9]]}

In [36]:
# Prepare the inference Requests
inference_request = {
    "dataframe_split": 
        test_df_6_input.to_dict(orient ='split')
}

In [37]:
# Send a prediction request to the locally depoloyed model

endpoint = "http://127.0.0.1:5000/invocations"
response = requests.post(endpoint, json=inference_request)

# Check if the response is successful
if response.status_code == 200:
    # Process the prediction response
    predictions = pd.DataFrame(response.json()['predictions'], columns=['Predicted Wine Quality'])

    # Combine predictions with actual classes
    actual_class_test = test_df_6['best quality'].reset_index(drop=True)
    model_output = pd.concat([predictions, actual_class_test], axis=1)

    # Rename columns for clarity
    model_output.columns = ['Predicted Wine Quality', 'Actual Wine Quality']

    # Display the final output
    print(model_output)
    
else:
    print(f"Request failed with status code: {response.status_code}")
    print(f"Response content: {response.text}")

   Predicted Wine Quality  Actual Wine Quality
0                       0                    1
1                       1                    1
2                       1                    1
3                       1                    1
4                       0                    0
5                       1                    1


Now, stop the mlflow model server with ***"crtl+c"***

<a id="model-packaging"></a>
# Model Packaging using Docker

**!!! Attention !!!** 

When **Dockerfile** is generated, this line should be added to be able to deploy the image onto OpenShift cluster since only **non-root Containers** are allowed to be deployed. 

Add this line:

__*"RUN chgrp -R 0 /opt && chmod -R g=u /opt"*__

after this line:

__*"RUN chmod o+rwX /opt/mlflow/"*__

In [38]:
# Generate Dockerfile and dependencies to package the model 
!mlflow models generate-dockerfile -m "{MODEL_URI}" --env-manager local -d "wine_clf_package_modelver_{MODEL_VERSION}" --enable-mlserver

2025/01/28 14:57:53 INFO mlflow.models.cli: Generating Dockerfile for model models:/ElasticnetWineModel/3

2025/01/28 14:57:55 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'

2025/01/28 14:57:55 INFO mlflow.models.cli: Generated Dockerfile in directory wine_clf_package_modelver_3


<a id="remote-model-server"></a>
# MLFlow Model Server on OpenShift

### It is time to build and deploy the model onto the OpenShift Clsuter

1. First push this new generated directory with Dockerfile and model artifacts to the preferred git (GitHub / GitLab / Azure DevOps / ...) repository. 
2. Then we go to OpenShift console to start a Build Process to package the model in an Image
3. Deploy the Image onto the OpenShift cluster. 
4. We come back to this notebook to test the deployed model on the cluster.

In [30]:
# Send a prediction request to the depoloyed model on the OpenShift Cluster
BASE_URL = "http://test-clf-app-test.apps.cluster-db46l.dynamic.redhatworkshops.io"
endpoint = f"{BASE_URL}/invocations"
response = requests.post(endpoint, json=inference_request)

# Check if the response is successful
if response.status_code == 200:
    # Process the prediction response
    predictions = pd.DataFrame(response.json()['predictions'], columns=['Predicted Wine Quality'])

    # Combine predictions with actual classes
    actual_class_test = test_df_6['best quality'].reset_index(drop=True)
    model_output = pd.concat([predictions, actual_class_test], axis=1)

    # Rename columns for clarity
    model_output.columns = ['Predicted Wine Quality', 'Actual Wine Quality']

    # Display the final output
    print(model_output)
    
else:
    print(f"Request failed with status code: {response.status_code}")
    print(f"Response content: {response.text}")

   Predicted Wine Quality  Actual Wine Quality
0                       0                    0
1                       1                    1
2                       1                    1
3                       0                    1
4                       1                    1
5                       1                    1


<<<<< IGNORE BELOW CELLs >>>>>

<<<<< The notebook will be further developed >>>>>

In [ ]:
# !set MLFLOW_TRACKING_URL="http://mlflow-mlflow.apps.cluster-db46l.dynamic.redhatworkshops.io"
# !set MLFLOW_TRACKING_USERNAME=user
# !set MLFLOW_TRACKING_PASSWORD=user
import os

# os.environ["MLFLOW_TRACKING_URI"] = "http://mlflow-mlflow.apps.cluster-db46l.dynamic.redhatworkshops.io"
# os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:8000"
# os.env[]